In [2]:
import pandas as pd
import numpy as np

### Question 1
You will be supplied with two data files in CSV format . The first file contains statistics about various dinosaurs. The second file contains additional data. Given the following formula, speed = ((STRIDE_LENGTH / LEG_LENGTH) - 1) * SQRT(LEG_LENGTH * g) Where g = 9.8 m/s^2 (gravitational constant)

Write a program to read in the data files from disk, it must then print the names of only the bipedal dinosaurs from fastest to slowest. Do not print any other information.


`$ cat dataset1.csv NAME,LEG_LENGTH,DIET Hadrosaurus,1.4,herbivore Struthiomimus,0.72,omnivore Velociraptor,1.8,carnivore Triceratops,0.47,herbivore Euoplocephalus,2.6,herbivore Stegosaurus,1.50,herbivore Tyrannosaurus Rex,6.5,carnivore`

`$ cat dataset2.csv NAME,STRIDE_LENGTH,STANCE Euoplocephalus,1.97,quadrupedal Stegosaurus,1.70,quadrupedal Tyrannosaurus Rex,4.76,bipedal Hadrosaurus,1.3,bipedal Deinonychus,1.11,bipedal Struthiomimus,1.24,bipedal Velociraptorr,2.62,bipedal`

In [3]:
df = pd.DataFrame

In [47]:
df10 = pd.DataFrame

In [48]:
d1,d2 = pd.read_csv('dataset1.csv'),pd.read_csv('dataset2.csv')

In [49]:
d1, d2

(                NAME  LEG_LENGTH       DIET
 0        Hadrosaurus        1.40  herbivore
 1      Struthiomimus        0.72   omnivore
 2       Velociraptor        1.80  carnivore
 3        Triceratops        0.47  herbivore
 4     Euoplocephalus        2.60  herbivore
 5        Stegosaurus        1.50  herbivore
 6  Tyrannosaurus Rex        6.50  carnivore,
                 NAME  STRIDE_LENGTH       STANCE
 0     Euoplocephalus           1.97  quadrupedal
 1        Stegosaurus           1.70  quadrupedal
 2  Tyrannosaurus Rex           4.76      bipedal
 3        Hadrosaurus           1.30      bipedal
 4        Deinonychus           1.11      bipedal
 5      Struthiomimus           1.24      bipedal
 6      Velociraptorr           2.62      bipedal)

speed = ((STRIDE_LENGTH / LEG_LENGTH) - 1) * SQRT(LEG_LENGTH * g) Where g = 9.8 m/s^2 (gravitational constant)

In [51]:
df10 = d1.merge(d2,on='NAME').copy()

In [52]:
df10

,NAME,LEG_LENGTH,DIET,STRIDE_LENGTH,STANCE
0,Hadrosaurus,1.40,herbivore,1.30,bipedal
1,Struthiomimus,0.72,omnivore,1.24,bipedal
2,Euoplocephalus,2.60,herbivore,1.97,quadrupedal
3,Stegosaurus,1.50,herbivore,1.70,quadrupedal
4,Tyrannosaurus Rex,6.50,carnivore,4.76,bipedal


In [150]:
g=9.8
df10['speed'] = (df10.STRIDE_LENGTH / df10.LEG_LENGTH) * np.sqrt(df10['LEG_LENGTH'] * g)

In [151]:
df10.sort_values('speed',ascending=False)

,NAME,LEG_LENGTH,DIET,STRIDE_LENGTH,STANCE,speed
4,Tyrannosaurus Rex,6.50,carnivore,4.76,bipedal,5.844715
1,Struthiomimus,0.72,omnivore,1.24,bipedal,4.574762
3,Stegosaurus,1.50,herbivore,1.70,quadrupedal,4.345266
2,Euoplocephalus,2.60,herbivore,1.97,quadrupedal,3.824658
0,Hadrosaurus,1.40,herbivore,1.30,bipedal,3.439477


In [152]:
df10[df10['STANCE'] == 'bipedal'].sort_values('speed',ascending=False)['NAME']

4    Tyrannosaurus Rex
1        Struthiomimus
0          Hadrosaurus
Name: NAME, dtype: object

In [ ]:
### how to do this with regular python


### Question 2:
Given an array of integers greater than zero, find if it is possible to split it in two subarrays (without reordering the elements), such that the sum of the two subarrays is the same. Print the two subarrays.

I calculated rightSum initially by using sum(nums), then looped over nums, and added nums[i] to leftSum, and subtracted it from rightSum, then checked if there was a match and printed that.

In [59]:
import random

In [60]:
myarr = [random.randint(1,4096) for x in range(10)]

In [61]:
myarr

[329, 625, 3481, 2143, 1034, 2190, 3080, 344, 155, 1093]

In [66]:
m = int(len(myarr)/2)

In [68]:
left = myarr[:m]
right = myarr[m:]

In [69]:
sum(left) == sum(right)

False

In [136]:
def splitArraysSame(arr):
    """
    This only covers the case for an even split of the elements
    """
    m = int(len(arr)/2)
    left = arr[:m]
    right = arr[m:]
    return sum(left) == sum(right)

In [78]:
# an array of length 10 can split into how many comparatives?
# 1/9, 2/8, 3/7, 4/6, 5/5, 6/4, 7/3, 8/2, 9/1 - 9 total combinations
# loop through - compare x + loop?
for x in range(0,len(myarr) -1):
    print(sum(myarr[:x]) == sum(myarr[x:]))

False
False
False
False
False
False
False
False
False


In [82]:
# split down the middle method should work
myarr2 = [1,2,3,4,5,5,4,3,2,1]
len(myarr2)

10

In [101]:
sum(myarr2[:5]) == sum(myarr2[5:])

True

In [96]:
# :2 == 2:
myarr3 = [5,5,1,1,1,1,1,1,2,2]
len(myarr3)

10

In [100]:
sum(myarr3[:2]) == sum(myarr3[2:])

True

In [140]:
splitArraysSame(myarr2)

True

In [102]:
# loop through - compare x + loop?
for x in range(0,len(myarr2) -1):
    print(sum(myarr2[:x]) == sum(myarr2[x:]))

False
False
False
False
False
True
False
False
False


In [103]:
# loop through - compare x + loop?
for x in range(0,len(myarr3) -1):
    print(sum(myarr3[:x]) == sum(myarr3[x:]))

False
False
True
False
False
False
False
False
False


In [109]:
def iterateArraySame(arr):
    """
    This works. Can I turn it into a lambda?
    """
    p=0
    for x in range(0,len(arr) -1):
        if sum(arr[:x]) == sum(arr[x:]):
            p+=1
    if p>0:
        return True
    else:
        return False


In [113]:
iterateArraySame(myarr3)

True

In [141]:
def lambdaArraySame(arr):
    xx=[]
    xx.append(lambda x: sum(arr[:x]) == sum(arr[x:]) for x in range(0,len(arr) -1))
    return xx

In [143]:
lambdaArraySame(myarr2)

[<generator object lambdaArraySame.<locals>.<genexpr> at 0x119ec0dd0>]


### Question 3:

There is a matrix with . and X, where X represents battleship, always of length 3. Battleship can be vertical or horizontal, never diagonal. Given a function bomb_at(i,j), returns True if battleship is present at (i,j) in the matrix. Print the head, middle, tail coordinates of the battleship.

In [183]:
# test harness
matrix = [['.','.','.','.','.'],['.','.','.','X','.'],['.','.','.','X','.'],['.','.','.','X','.'],['.','.','.','.','.']]

In [154]:
matrix

[['.', '.', '.', '.', '.'],
 ['.', '.', '.', 'X', '.'],
 ['.', '.', '.', 'X', '.'],
 ['.', '.', '.', 'X', '.'],
 ['.', '.', '.', '.', '.']]

In [176]:
import itertools

In [173]:
# test the iterator
battleship={}
for i, j in itertools.product(range(5), range(5)):
    if matrix[i][j] == 'X':
        battleship[i]=j
print(battleship)
    

{1: 3, 2: 3, 3: 3}


In [179]:
# create a test function bomb_at
def bomb_at(mat,i,j):
    if mat[i][j] =='X':
        return True
    

In [186]:
# 
size=len(matrix)
bombs=[]
for i, j in itertools.product(range(size), range(size)):
    if bomb_at(matrix,i,j):
        bombs.append((i,j))
if len(bombs) > 2 and len(bombs) < 4:
    print(bombs)
    

[(1, 3), (2, 3), (3, 3)]


In [199]:
# test harness for no bombs
matrix1= [['.','.','.','.','.'],['.','.','.','.','.'],['.','.','.','.','.'],['.','.','.','.','.'],['.','.','.','.','.']]

In [200]:
# test harness for too many bombs
matrix2= [['.','.','.','X','.'],['.','.','.','X','.'],['.','.','.','X','.'],['.','.','.','X','.'],['.','.','.','.','.']]

In [201]:
# test harness for too few bombs
matrix3= [['.','.','.','.','.'],['.','.','.','.','.'],['.','.','.','X','.'],['.','.','.','X','.'],['.','.','.','.','.']]

In [202]:
mat = matrix1
size=len(mat)
bombs=[]
for i, j in itertools.product(range(size), range(size)):
    if bomb_at(mat,i,j):
        bombs.append((i,j))
if len(bombs) > 2 and len(bombs) < 4:
    print(bombs)
else:
    print("no bombs or wrong number of bombs")

no bombs or wrong number of bombs


In [206]:
# extend to show no or some bombs
mat = matrix3
size=len(mat)
bombs=[]
for i, j in itertools.product(range(size), range(size)):
    if bomb_at(mat,i,j):
        bombs.append((i,j))
if len(bombs) > 2 and len(bombs) < 4:
    print(bombs)
elif len(bombs) > 0 and len(bombs) < 3:
    print('not enough bombs to matter')
elif len(bombs) > 3:
    print('too many bombs to worry about')
else:
    print("no bombs")

not enough bombs to matter
